In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-email-detection/Spam Email Detection.xlsx


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_excel('/kaggle/input/spam-email-detection/Spam Email Detection.xlsx')
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
# Concatenate the three columns and create a new column 'Concatenated'
df = df.fillna('').astype(str)

# Concatenate the columns
df['Message'] = df['v2'].fillna('') + df['Unnamed: 2'].fillna('') + df['Unnamed: 3'].fillna('') + df['Unnamed: 4'].fillna('')

# Print the resulting DataFrame
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,Message
0,ham,"Go until jurong point, crazy.. Available only ...",,,,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...,,,,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,,,,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,,,,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",,,,"Nah I don't think he goes to usf, he lives aro..."
...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,,,,This is the 2nd time we have tried 2 contact u...
5568,ham,Will �_ b going to esplanade fr home?,,,,Will �_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s...",,,,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...,,,,The guy did some bitching but I acted like i'd...


In [5]:
# Drop columns
df.drop(['v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)
df.head()

,v1,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# Rename columns
df.columns = ['Category', 'Message']
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [8]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [9]:
# Adding New column - Spam (0 - Not spam or Ham and 1 - Spam )

df['Spam']=df['Category'].apply(lambda x:1 if x=='spam' else 0)
df.head(5)

,Category,Message,Spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [10]:
df['Spam'].value_counts()

# Thus there are 4825 Ham (Not Spam Mails) and 747 Spam Mails

0    4825
1     747
Name: Spam, dtype: int64

## Modelling

In [11]:
X = df['Message']
y = df['Spam']

In [12]:
# Splitting the data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=0)

In [17]:
print(X_train.shape)
print(X_test.shape)

(4457,)
(1115,)


In [18]:
# Converting into TF-IDF Matrix

from sklearn.feature_extraction.text import TfidfVectorizer

# Removing stopwords
tfidf = TfidfVectorizer(stop_words='english')


# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix_train = tfidf.fit_transform(X_train)
tfidf_matrix_test= tfidf.transform(X_test)

print(tfidf_matrix_train.shape)
print(tfidf_matrix_test.shape)

(4457, 7397)
(1115, 7397)


#### 1. Logistic Regression

In [30]:
# Training the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score

log_reg_classifier = LogisticRegression()
log_reg_classifier.fit(tfidf_matrix_train, y_train)

# Prediction
y_pred_log = log_reg_classifier.predict(tfidf_matrix_test)

# Evaluation
acc = accuracy_score(y_test, y_pred_log)
precision = precision_score(y_test, y_pred_log)
recall = recall_score(y_test, y_pred_log)
f1 = f1_score(y_test, y_pred_log)

print("Accuracy : {}\nPrecision : {}\nRecall : {}\nF1 Score : {}".format(acc, precision, recall, f1))

Accuracy : 0.9587443946188341
Precision : 0.9918032786885246
Recall : 0.7289156626506024
F1 Score : 0.8402777777777779


In [26]:
report = classification_report(y_test, y_pred_log) 
print(report)

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       949
           1       0.99      0.73      0.84       166

    accuracy                           0.96      1115
   macro avg       0.97      0.86      0.91      1115
weighted avg       0.96      0.96      0.96      1115



#### 2. KNN 

In [32]:
# Fitting the KMM model
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p=1)

knn_classifier.fit(tfidf_matrix_train, y_train)

# Prediction
y_pred_knn = knn_classifier.predict(tfidf_matrix_test)

# Evaluation
acc = accuracy_score(y_test, y_pred_knn)
precision = precision_score(y_test, y_pred_knn)
recall = recall_score(y_test, y_pred_knn)
f1 = f1_score(y_test, y_pred_knn)

print("Accuracy : {}\nPrecision : {}\nRecall : {}\nF1 Score : {}".format(acc, precision, recall, f1))

Accuracy : 0.8556053811659193
Precision : 1.0
Recall : 0.030120481927710843
F1 Score : 0.058479532163742694


#### 3. Random Forest Classifier

In [33]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

rfc.fit(tfidf_matrix_train, y_train)

# Prediction
y_pred_rfc = rfc.predict(tfidf_matrix_test)

# Evaluation
acc = accuracy_score(y_test, y_pred_rfc)
precision = precision_score(y_test, y_pred_rfc)
recall = recall_score(y_test, y_pred_rfc)
f1 = f1_score(y_test, y_pred_rfc)

print("Accuracy : {}\nPrecision : {}\nRecall : {}\nF1 Score : {}".format(acc, precision, recall, f1))

Accuracy : 0.9721973094170404
Precision : 1.0
Recall : 0.8132530120481928
F1 Score : 0.8970099667774087


#### 4. SVC

In [34]:
# Training
from sklearn.svm import SVC

svc = SVC(kernel='rbf', C=10)

svc.fit(tfidf_matrix_train, y_train)

# Prediction
y_pred_svc = svc.predict(tfidf_matrix_test)

# Evaluation
acc = accuracy_score(y_test, y_pred_svc)
precision = precision_score(y_test, y_pred_svc)
recall = recall_score(y_test, y_pred_svc)
f1 = f1_score(y_test, y_pred_svc)

print("Accuracy : {}\nPrecision : {}\nRecall : {}\nF1 Score : {}".format(acc, precision, recall, f1))

Accuracy : 0.9811659192825112
Precision : 0.9931972789115646
Recall : 0.8795180722891566
F1 Score : 0.9329073482428114
